In [6]:
import pandas as pd
import minsearch

In [5]:
df = pd.read_csv('../data/data.csv')
df.head()

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."


In [10]:
documents = df.to_dict(orient='records')

In [ ]:
docum

In [7]:
df.columns

Index(['id', 'exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [8]:
index = minsearch.Index(
    text_fields=['exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
       keyword_fields=[]
)

In [11]:
index.fit(documents)

In [13]:
query = "What exercises can I do for my biceps at home?"

In [14]:
index.search(query, num_results=5)

[{'id': 152,
  'exercise_name': 'Machine Preacher Curl',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Machine',
  'body_part': 'Upper Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Biceps',
  'instructions': 'Sit at a preacher curl machine with your arms on the pad. Curl the handle towards your shoulders, then lower it back down.'},
 {'id': 57,
  'exercise_name': 'Single Arm Dumbbell Row',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Dumbbell',
  'body_part': 'Upper Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Latissimus Dorsi, Biceps',
  'instructions': 'Place one knee and hand on a bench for support, row the dumbbell towards your torso with the other hand.'},
 {'id': 150,
  'exercise_name': 'One-Arm Dumbbell Row',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Dumbbell',
  'body_part': 'Upper Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Latissimus Dorsi, Biceps',
  'instructions': 'Place one knee and hand on a bench for

In [16]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = openai.Client(api_key=api_key)


In [26]:
def search(query):
    boost = {}

    results = index.search(
        query = query,
        filter_dict = {},
        boost_dict = boost,
        num_results = 5
    ) 
    return results

In [27]:
prompt_template = """
You're a fitness insrtuctor. Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
exercise_name: {exercise_name}
type_of_activity: {type_of_activity}
type_of_equipment: {type_of_equipment}
body_part: {body_part}
type: {type}
muscle_groups_activated: {muscle_groups_activated}
instructions: {instructions}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [28]:
def llm(prompt):
    response=client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{"role": "user", "content": prompt}],)

    return response.choices[0].message.content

In [29]:
def rag(query):
    search_results = search(query)
    prompt=build_prompt(query, search_results)
    answer=llm(prompt)
    return answer

In [30]:
answer = rag(query)
print(answer)

Here are the biceps-focused exercises from the context that you can do at home with dumbbells:

- Single Arm Dumbbell Row
  - Equipment: dumbbell
  - How it activates: Biceps (along with latissimus dorsi)

- One-Arm Dumbbell Row
  - Equipment: dumbbell
  - How it activates: Biceps (along with latissimus dorsi)

- Dumbbell Bent-Over Row
  - Equipment: dumbbells
  - How it activates: Biceps (along with latissimus dorsi)

Note: The other biceps-related exercises in the context (Machine Preacher Curl and Smith Machine Row) require machines, which may not be available at home.
